In [1]:
# pip install xgboost

In [2]:
# Data Manipulation
import numpy as np
import pandas as pd
# Data Visualization
import matplotlib.pyplot as plt
# Data Modelling
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import accuracy_score, precision_score, recall_score,\
f1_score, confusion_matrix, ConfusionMatrixDisplay, RocCurveDisplay
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from xgboost import XGBClassifier
from xgboost import XGBRegressor

# This is the function that helps plot feature importance 
from xgboost import plot_importance

In [3]:
# RUN THIS CELL TO SEE ALL COLUMNS 
# This lets us see all of the columns, preventing Juptyer from redacting them.
# pd.set_option('display.max_rows', None)

In [4]:
df=pd.read_csv(r"C:\Users\yorgh\Documents\Course Materials\Google Advanced Data Analytics Course\Datasets Exercise\Automatidata project\2017_Yellow_Taxi_Trip_Features_engineering(2).csv")

In [5]:
df

,trip_distance,duration,total_amount,rush_hour
0,3.34,14.066667,13.8,0
1,1.80,26.500000,16.8,0
2,1.00,7.200000,7.3,1
3,3.70,30.250000,21.3,0
4,4.37,16.716667,17.8,0
...,...,...,...,...
21917,0.89,9.450000,8.8,0
21918,0.61,3.266667,5.8,1
21919,0.42,4.133333,5.3,0
21920,2.36,11.933333,11.3,0


**Upload the clean data with necessary modifications**

In [6]:
# Isolate target variable (y)
y = df['total_amount']

# Isolate the features (X)
X = df.drop('total_amount', axis=1)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Split train into tr and val sets
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

**Train/test split of 20% and Train/validation split of 25%**

## Random Forest Regression Model:

In [7]:
# 1. Instantiate the random forest regressor
rf = RandomForestRegressor(random_state=42)

# 2. Create a dictionary of hyperparameters to tune 
cv_params = {'max_depth': [None],
             'max_features': [1.0],
             'max_samples': [0.7],
             'min_samples_leaf': [1],
             'min_samples_split': [2],
             'n_estimators': [300]
             }

# Define a set of scoring metrics to capture
scoring = {'r2': make_scorer(r2_score),
           'neg_mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False), # negate for GridSearchCV
           'neg_mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False), # negate for GridSearchCV
           'neg_root_mean_squared_error': make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False) # negate for GridSearchCV
          }

# 4. Instantiate the GridSearchCV object
rf_grid = GridSearchCV(rf, cv_params, scoring=scoring, cv=3, refit='r2')

**Please note that these hyperparameters have been obtained after many itirations of a different mix of hyperparameters**

In [8]:
%%time
rf_grid.fit(X_tr, y_tr)

CPU times: total: 11.2 s
Wall time: 12.7 s


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
             param_grid={'max_depth': [None], 'max_features': [1.0],
                         'max_samples': [0.7], 'min_samples_leaf': [1],
                         'min_samples_split': [2], 'n_estimators': [300]},
             refit='r2',
             scoring={'neg_mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
                      'neg_mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False),
                      'neg_root_mean_squared_error': make_scorer(<lambda>, greater_is_better=False),
                      'r2': make_scorer(r2_score)})

In [9]:
# Examine best score
rf_grid.best_score_

0.9799422137283056

In [12]:
rf_grid.best_params_

{'max_depth': None,
 'max_features': 1.0,
 'max_samples': 0.7,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 300}

In [13]:
def make_results(model_name:str, model_object, metric:str):
    '''
    Arguments:
    model_name (string): what you want the model to be called in the output table
    model_object: a fit GridSearchCV object
    metric (string): r2, MAE, MSE, or RMSE

    Returns a pandas df with the R-squared, MAE, MSE, and RMSE scores
    for the model with the best mean 'metric' score across all validation folds.
    '''

    # Create dictionary that maps input metric to actual metric name in GridSearchCV
    metric_dict = {'r2': 'mean_test_r2',
                   'MAE': 'mean_test_neg_mean_absolute_error',
                   'MSE': 'mean_test_neg_mean_squared_error',
                   'RMSE': 'mean_test_neg_root_mean_squared_error',
                   }

    # Get all the results from the CV and put them in a df
    cv_results = pd.DataFrame(model_object.cv_results_)

    # Isolate the row of the df with the max(metric) score
    best_estimator_results = cv_results.iloc[cv_results[metric_dict[metric]].idxmax(), :]

    # Extract R-squared, MAE, MSE, and RMSE scores from that row
    r2 = best_estimator_results.mean_test_r2
    MAE = -best_estimator_results.mean_test_neg_mean_absolute_error  # negate back
    MSE = -best_estimator_results.mean_test_neg_mean_squared_error  # negate back
    RMSE = -best_estimator_results.mean_test_neg_root_mean_squared_error  # negate back

    # Create table of results
    table = pd.DataFrame({'model': [model_name],
                          'R-squared': [r2],
                          'MAE': [MAE],
                          'MSE': [MSE],
                          'RMSE': [RMSE],
                         })

    return table

In [14]:
# Call 'make_results()' on the GridSearch object
results = make_results('RF CV', rf_grid, 'r2')
results

,model,R-squared,MAE,MSE,RMSE
0,RF CV,0.979942,0.527675,1.373694,1.171614


## XG boost Regression Model:

In [15]:
# 1. Instantiate the XGBoost regressor
xgb = XGBRegressor(objective='reg:squarederror', random_state=0)

# 2. Create a dictionary of hyperparameters to tune 
cv_params = {'learning_rate': [0.1,0.3,0.5],
             'max_depth': [None,10,7],
             'min_child_weight': [2,1,3],
             'n_estimators': [300]
             }

# 3. Define a set of scoring metrics to capture
scoring = {'r2': make_scorer(r2_score),
           'neg_mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False), # negate for GridSearchCV
           'neg_mean_squared_error': make_scorer(mean_squared_error, greater_is_better=False), # negate for GridSearchCV
           'neg_root_mean_squared_error': make_scorer(lambda y_true, y_pred: mean_squared_error(y_true, y_pred, squared=False), greater_is_better=False) # negate for GridSearchCV
          }

# 4. Instantiate the GridSearchCV object
xgb1 = GridSearchCV(xgb, cv_params, scoring=scoring, cv=3, refit='r2')

In [16]:
%%time
xgb1.fit(X_tr, y_tr)

CPU times: total: 2min 9s
Wall time: 24.2 s


GridSearchCV(cv=3,
             estimator=XGBRegressor(base_score=None, booster=None,
                                    callbacks=None, colsample_bylevel=None,
                                    colsample_bynode=None,
                                    colsample_bytree=None, device=None,
                                    early_stopping_rounds=None,
                                    enable_categorical=False, eval_metric=None,
                                    feature_types=None, gamma=None,
                                    grow_policy=None, importance_type=None,
                                    interaction_constraints=None,
                                    learning_rate=None, m...
                         'max_depth': [None, 10, 7],
                         'min_child_weight': [2, 1, 3], 'n_estimators': [300]},
             refit='r2',
             scoring={'neg_mean_absolute_error': make_scorer(mean_absolute_error, greater_is_better=False),
                      'neg

In [17]:
# Examine best score
xgb1.best_score_

0.9758201045676627

In [18]:
# Examine best parameters
xgb1.best_params_

{'learning_rate': 0.1,
 'max_depth': None,
 'min_child_weight': 1,
 'n_estimators': 300}

## Results:

In [19]:
# Call 'make_results()' on the GridSearch object
xgb1_cv_results = make_results('XGB CV', xgb1, 'r2')
results = pd.concat([results, xgb1_cv_results], axis=0)
results

,model,R-squared,MAE,MSE,RMSE
0,RF CV,0.979942,0.527675,1.373694,1.171614
0,XGB CV,0.975820,0.547007,1.654026,1.286033


**Random Forest gave better results**

1) `R²: The coefficient of determination`

R² measures the proportion of variation in the dependent variable, Y, explained by the independent variable(s), X.

•	This is calculated by subtracting the sum of squared residuals divided by the total sum of squares from 1.
R²=1−Sum of Total sum of squared residuals
R² ranges from 0 to 1. So, if a model has an R² of 0.85, that means that the X variables explain about 85% of the variation in the Y variable. Although R² is a highly interpretable and commonly used metric, you may also encounter mean squared error (MSE) and mean absolute error (MAE) when R² is insufficient in evaluating model performance.

2) `MSE: Mean squared error`

MSE (mean squared error) is the average of the squared difference between the predicted and actual values.
•	Because of how MSE is calculated, MSE is very sensitive to large errors.

3) `MAE: Mean absolute error`

MAE (mean absolute error) is the average of the absolute difference between the predicted and actual values.
•	If your data has outliers that you want to ignore, you can use MAE, as it is not sensitive to large errors.

4) `Adjusted R²`

The adjusted R-squared penalizes the addition of more independent variables to the multiple regression model. Additionally, the adjusted R-squared only captures the proportion of variation explained by the independent variables that show a significant relationship with the outcome variable. 

5) `RMSE: Root Mean squared error`

RMSE is simply the square root of the MSE. It is particularly useful because it's in the same units as the target variable, making it easier to interpret.

## Multi-base-learner Regression Model:

In [20]:
from sklearn.model_selection import cross_val_predict
# Define base learners
base_learners = {
    "Linear Regression": (LinearRegression(), None),
    "Random Forest": (RandomForestRegressor(random_state=42), {'n_estimators': [100, 200, 300],
                                                              'max_depth': [None, 10, 20]}),
    "XGBoost": (XGBRegressor(objective='reg:squarederror', random_state=42), None)
}

# Train and evaluate each base learner
predictions = {}
for name, (model, param_grid) in base_learners.items():
    # Perform hyperparameter tuning if param_grid is not empty
    if param_grid:
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        grid_search.fit(X_tr, y_tr)
        model = grid_search.best_estimator_
    
    # Train the model
    model.fit(X_tr, y_tr)
    
    # Evaluate the model
    if name == "Linear Regression":
        # For linear regression, we use cross-validation
        y_pred = cross_val_predict(model, X_tr, y_tr, cv=5)
        mse = mean_squared_error(y_tr, y_pred)
    else:
        # For other models, we use holdout set
        y_pred = model.predict(X_val)
        mse = mean_squared_error(y_val, y_pred)
    
    predictions[name] = {'predictions': y_pred, 'mse': mse}

# Select the best performing model
best_model_name = min(predictions, key=lambda k: predictions[k]['mse'])
best_model_predictions = predictions[best_model_name]['predictions']
best_model_mse = predictions[best_model_name]['mse']

print("Best performing model:", best_model_name)
print("MSE of best performing model:", best_model_mse)

Best performing model: Random Forest
MSE of best performing model: 1.4115460492959693


**The purpose of this multi-base-learner function is to iterate through various regression models.
By leveraging predefined scoring metrics and assuming well-tuned hyperparameters, this approach allows us to identify the best-performing model among different types of regression models.**

In [28]:
# Define base learners with hyperparameter grids for tuning
base_learners = {
    "Linear Regression": (LinearRegression(), {}),
    "Random Forest": (RandomForestRegressor(random_state=42), {'n_estimators': [100, 200, 300],
                                                               'max_depth': [None, 10, 20]}),
    "XGBoost": (XGBRegressor(objective='reg:squarederror', random_state=42), {'n_estimators': [100, 200, 300],
                                                                              'max_depth': [3, 5, 7],
                                                                              'learning_rate': [0.1, 0.01]})
}

# Define a function to calculate RMSE
def calculate_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Initialize a list to store the results
results = []

# Train and evaluate each base learner
for name, (model, param_grid) in base_learners.items():
    # Perform hyperparameter tuning if param_grid is not empty
    if param_grid:
        grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
        grid_search.fit(X_tr, y_tr)
        best_model = grid_search.best_estimator_
        best_params = grid_search.best_params_
    else:
        best_model = model
        best_params = None
    
    # Train the model
    best_model.fit(X_tr, y_tr)
    
    # Evaluate the model on train data
    y_train_pred = best_model.predict(X_tr)
    r2_train = r2_score(y_tr, y_train_pred)
    mae_train = mean_absolute_error(y_tr, y_train_pred)
    mse_train = mean_squared_error(y_tr, y_train_pred)
    rmse_train = calculate_rmse(y_tr, y_train_pred)
    
    # Evaluate the model on test data
    y_test_pred = best_model.predict(X_val)
    r2_test = r2_score(y_val, y_test_pred)
    mae_test = mean_absolute_error(y_val, y_test_pred)
    mse_test = mean_squared_error(y_val, y_test_pred)
    rmse_test = calculate_rmse(y_val, y_test_pred)
    
    # Store results in the list
    results.append({
        'Model': name,
        'Best Parameters': best_params,
        'R-squared Train': r2_train,
        'MAE Train': mae_train,
        'MSE Train': mse_train,
        'RMSE Train': rmse_train,
        'R-squared Test': r2_test,
        'MAE Test': mae_test,
        'MSE Test': mse_test,
        'RMSE Test': rmse_test
    })

# Convert the list of results into a DataFrame
results_df = pd.DataFrame(results)

# Ensure the MSE columns are numeric
results_df['MSE Train'] = pd.to_numeric(results_df['MSE Train'])
results_df['MSE Test'] = pd.to_numeric(results_df['MSE Test'])

# # Print best performing model on train data
# best_train_model = results_df.loc[results_df['MSE Train'].idxmin()]
# print("Best performing model on train data:")
# print(best_train_model)

# Print best performing model on test data
best_test_model = results_df.loc[results_df['MSE Test'].idxmin()]
print("\nBest performing model on test data:")
print(best_test_model)

# Print results table with better formatting
print("\nResults:")
print(results_df.to_string(index=False))


Best performing model on test data:
Model                                       Random Forest
Best Parameters    {'max_depth': 10, 'n_estimators': 300}
R-squared Train                                  0.992325
MAE Train                                        0.390374
MSE Train                                        0.525415
RMSE Train                                       0.724855
R-squared Test                                   0.980358
MAE Test                                         0.505911
MSE Test                                         1.411546
RMSE Test                                        1.188085
Name: 1, dtype: object

Results:
            Model                                             Best Parameters  R-squared Train  MAE Train  MSE Train  RMSE Train  R-squared Test  MAE Test  MSE Test  RMSE Test
Linear Regression                                                        None         0.978263   0.582476   1.488010    1.219840        0.978860  0.564972  1.519166   1.23254

**Summary table**

In [ ]:
print(results_df['MSE Train'].head())
print(results_df['MSE Train'].dtype)